In [1]:
"""

https://www.kaggle.com/osciiart/advanced-basketball-analytics-with-ensembling
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# %matplotlib inline
from math import pi
# import seaborn as sns
import time

from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn import preprocessing, metrics,ensemble, model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from xgboost.sklearn import XGBClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, classification_report, confusion_matrix

pd.set_option('display.max_columns', 999)
pd.options.display.float_format = '{:.6f}'.format

start_time = time.time()

In [5]:
#standard files

#df_tourney = pd.read_csv('NCAATourneyCompactResults.csv')
#df_season = pd.read_csv('RegularSeasonDetailedResults.csv')
#df_teams = pd.read_csv('Teams.csv')
#df_seeds = pd.read_csv('NCAATourneySeeds.csv')
#df_conferences = pd.read_csv('Conferences.csv')
#df_rankings = pd.read_csv('MasseyOrdinals.csv')
#df_sample_sub = pd.read_csv('SampleSubmissionStage1.csv')
#df_team_conferences = pd.read_csv('Teamconferences.csv')
#df_ConferenceTourneyGames = pd.read_csv('ConferenceTourneyGames.csv')

#my custom file


#df_tourney_experience = pd.read_csv('tourney_experience_senior_class.csv')

# Kaggle locations

df_tourney = pd.read_csv('../Winput/WNCAATourneyCompactResults.csv')
df_season = pd.read_csv('../Winput/WRegularSeasonDetailedResults_PrelimData2018.csv') # TODO update
df_teams = pd.read_csv('../Winput/WTeams.csv')
df_seeds = pd.read_csv('../Winput/WNCAATourneySeeds_SampleTourney2018.csv') # TODO update
# df_conferences = pd.read_csv('../Winput/WConferences.csv')
# df_rankings = pd.read_csv('../Winput/WMasseyOrdinals_Prelim2018.csv') # TODO update
df_sample_sub = pd.read_csv('../Winput/WSampleSubmissionStage2_SampleTourney2018.csv') # TODO update
# df_team_conferences = pd.read_csv('../Winput/WTeamconferences.csv')

#private data file

df_tourney_experience = pd.read_csv('../additional/Wexperience.csv')

In [6]:
#Calculate Winning/losing Team Possesion Feature

#https://www.nbastuffer.com/analytics101/possession/


wPos = df_season.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
lPos = df_season.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)

#two teams use almost the same number of possessions in a game
#(plus/minus one or two - depending on how quarters end)
#so let's just take the average

df_season['Possesions'] = (wPos+lPos)/2

In [7]:
df_season.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Possesions
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19,67.564800
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27,76.416000
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15,63.552000
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14,72.460800
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14,72.998400


In [8]:
#Name Player Impact Estimate Definition PIE measures a player's overall statistical contribution
#against the total statistics in games they play in. PIE yields results which are
#comparable to other advanced statistics (e.g. PER) using a simple formula.
#Formula (PTS + FGM + FTM - FGA - FTA + DREB + (.5 * OREB) + AST + STL + (.5 * BLK) - PF - TO)
# / (GmPTS + GmFGM + GmFTM - GmFGA - GmFTA + GmDREB + (.5 * GmOREB) + GmAST + GmSTL + (.5 * GmBLK) - GmPF - GmTO)

#We will use this to measure Team Skill

wtmp = df_season.apply(lambda row: row.WScore + row.WFGM + row.WFTM - row.WFGA - row.WFTA + row.WDR + 0.5*row.WOR + row.WAst +row.WStl + 0.5*row.WBlk - row.WPF - row.WTO, axis=1)
ltmp = df_season.apply(lambda row: row.LScore + row.LFGM + row.LFTM - row.LFGA - row.LFTA + row.LDR + 0.5*row.LOR + row.LAst +row.LStl + 0.5*row.LBlk - row.LPF - row.LTO, axis=1) 

df_season['WPIE'] = wtmp/(wtmp + ltmp)
df_season['LPIE'] = ltmp/(wtmp + ltmp)

In [9]:
#Four factors statistic from the NBA

#https://www.nbastuffer.com/analytics101/four-factors/


#Effective Field Goal Percentage=(Field Goals Made) + 0.5*3P Field Goals Made))/(Field Goal Attempts)
#you have to put the ball in the bucket eventually

df_season['WeFGP'] = df_season.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
df_season['LeFGP'] = df_season.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1) 

#Turnover Rate= Turnovers/(Field Goal Attempts + 0.44*Free Throw Attempts + Turnovers)
#he who doesnt turn the ball over wins games

df_season['WTOR'] = df_season.apply(lambda row: row.WTO / (row.WFGA + 0.44*row.WFTA + row.WTO), axis=1)
df_season['LTOR'] = df_season.apply(lambda row: row.LTO / (row.LFGA + 0.44*row.LFTA + row.LTO), axis=1)


#Offensive Rebounding Percentage = (Offensive Rebounds)/[(Offensive Rebounds)+(Opponent’s Defensive Rebounds)]
#You can win games controlling the offensive glass

df_season['WORP'] = df_season.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
df_season['LORP'] = df_season.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)

#Free Throw Rate=(Free Throws Made)/(Field Goals Attempted) or Free Throws Attempted/Field Goals Attempted
#You got to get to the line to win close games

df_season['WFTAR'] = df_season.apply(lambda row: row.WFTA / row.WFGA, axis=1)
df_season['LFTAR'] = df_season.apply(lambda row: row.LFTA / row.LFGA, axis=1)

#4 Factors is weighted as follows
#1. Shooting (40%)
#2. Turnovers (25%)
#3. Rebounding (20%)
#4. Free Throws (15%)

df_season['W4Factor'] = df_season.apply(lambda row: .40*row.WeFGP + .25*row.WTOR + .20*row.WORP + .15*row.WFTAR, axis=1)
df_season['L4Factor'] = df_season.apply(lambda row: .40*row.LeFGP + .25*row.LTOR + .20*row.LORP + .15*row.LFTAR, axis=1)      

In [10]:
#Offensive efficiency (OffRtg) =  (Points / Possessions)
#Every possession counts

df_season['WOffRtg'] = df_season.apply(lambda row: (row.WScore / row.Possesions), axis=1)
df_season['LOffRtg'] = df_season.apply(lambda row: (row.LScore / row.Possesions), axis=1)

#Defensive efficiency (DefRtg) = (Opponent points / Opponent possessions)
#defense wins championships

df_season['WDefRtg'] = df_season.LOffRtg
df_season['LDefRtg'] = df_season.WOffRtg

                        
#Assist Ratio : Percentage of team possessions that end in assists
#distribute the rock - dont go isolation all the time

df_season['WAstR'] = df_season.apply(lambda row: row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
df_season['LAstR'] = df_season.apply(lambda row: row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)


#DREB% : Percentage of team defensive rebounds
#control your own glass

df_season['WDRP'] = df_season.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
df_season['LDRP'] = df_season.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1) 

#Free Throw Percentage
#Make your damn free throws

df_season['WFTPCT'] = df_season.apply(lambda row : 0 if row.WFTA < 1 else row.WFTM / row.WFTA, axis=1)
df_season['LFTPCT'] = df_season.apply(lambda row : 0 if row.LFTA < 1 else row.LFTM / row.LFTA, axis=1)

In [11]:
df_season.drop(['WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF'], axis=1, inplace=True)
df_season.drop(['LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'], axis=1, inplace=True)

In [12]:
df_season.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Possesions,WPIE,LPIE,WeFGP,LeFGP,WTOR,LTOR,WORP,LORP,WFTAR,LFTAR,W4Factor,L4Factor,WOffRtg,LOffRtg,WDefRtg,LDefRtg,WAstR,LAstR,WDRP,LDRP,WFTPCT,LFTPCT
0,2010,11,3103,63,3237,49,H,0,67.564800,0.661654,0.338346,0.472222,0.398148,0.223992,0.282555,0.270270,0.297297,0.351852,0.185185,0.351719,0.317135,0.932438,0.725230,0.725230,0.932438,0.148368,0.119048,0.702703,0.729730,0.631579,0.600000
1,2010,11,3104,73,3399,68,N,0,76.416000,0.727273,0.272727,0.459677,0.428571,0.212044,0.210793,0.380952,0.311111,0.451613,0.428571,0.380814,0.350635,0.955297,0.889866,0.889866,0.955297,0.137212,0.068708,0.688889,0.619048,0.571429,0.518519
2,2010,11,3110,71,3224,59,A,0,63.552000,0.648045,0.351955,0.516129,0.344828,0.161933,0.180462,0.378378,0.425000,0.193548,0.396552,0.351643,0.327529,1.117195,0.928374,0.928374,1.117195,0.183150,0.087796,0.575000,0.621622,0.583333,0.826087
3,2010,11,3111,63,3267,58,A,0,72.460800,0.632768,0.367232,0.557692,0.283784,0.325458,0.114583,0.214286,0.354839,0.173077,0.337838,0.373260,0.263803,0.869436,0.800433,0.800433,0.869436,0.144389,0.135135,0.645161,0.785714,0.555556,0.640000
4,2010,11,3119,74,3447,70,H,1,72.998400,0.591346,0.408654,0.452703,0.398649,0.122440,0.143974,0.304348,0.388889,0.148649,0.283784,0.294858,0.315798,1.013721,0.958925,0.958925,1.013721,0.166914,0.109850,0.611111,0.695652,0.636364,0.523810


In [13]:
df_season_composite = pd.DataFrame()

#This will aggregate individual games into season totals for a team

#calculates wins and losses to get winning percentage

df_season_composite['WINS'] = df_season['WTeamID'].groupby([df_season['Season'], df_season['WTeamID']]).count()
df_season_composite['LOSSES'] = df_season['LTeamID'].groupby([df_season['Season'], df_season['LTeamID']]).count()
df_season_composite['WINPCT'] = df_season_composite['WINS'] / (df_season_composite['WINS'] + df_season_composite['LOSSES'])

# calculates averages for games team won

df_season_composite['WPIE'] = df_season['WPIE'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WeFGP'] = df_season['WeFGP'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WTOR'] = df_season['WTOR'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WORP'] = df_season['WORP'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WFTAR'] = df_season['WFTAR'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['W4Factor'] = df_season['W4Factor'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WOffRtg'] = df_season['WOffRtg'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WDefRtg'] = df_season['WDefRtg'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WAstR'] = df_season['WAstR'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WDRP'] = df_season['WDRP'].groupby([df_season['Season'], df_season['WTeamID']]).mean()
df_season_composite['WFTPCT'] = df_season['WFTPCT'].groupby([df_season['Season'], df_season['WTeamID']]).mean()

# calculates averages for games team lost

df_season_composite['LPIE'] = df_season['LPIE'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LeFGP'] = df_season['LeFGP'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LTOR'] = df_season['LTOR'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LORP'] = df_season['LORP'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LFTAR'] = df_season['LFTAR'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['L4Factor'] = df_season['L4Factor'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LOffRtg'] = df_season['LOffRtg'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LDefRtg'] = df_season['LDefRtg'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LAstR'] = df_season['LAstR'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LDRP'] = df_season['LDRP'].groupby([df_season['Season'], df_season['LTeamID']]).mean()
df_season_composite['LFTPCT'] = df_season['LFTPCT'].groupby([df_season['Season'], df_season['LTeamID']]).mean()

# calculates weighted average using winning percent to weight the statistic


df_season_composite['PIE'] = df_season_composite['WPIE'] * df_season_composite['WINPCT'] + df_season_composite['LPIE'] * (1 - df_season_composite['WINPCT'])
df_season_composite['FG_PCT'] = df_season_composite['WeFGP'] * df_season_composite['WINPCT'] + df_season_composite['LeFGP'] * (1 - df_season_composite['WINPCT'])
df_season_composite['TURNOVER_RATE'] = df_season_composite['WTOR'] * df_season_composite['WINPCT'] + df_season_composite['LTOR'] * (1 - df_season_composite['WINPCT'])
df_season_composite['OFF_REB_PCT'] = df_season_composite['WORP'] * df_season_composite['WINPCT'] + df_season_composite['LORP'] * (1 - df_season_composite['WINPCT'])
df_season_composite['FT_RATE'] = df_season_composite['WFTAR'] * df_season_composite['WINPCT'] + df_season_composite['LFTAR'] * (1 - df_season_composite['WINPCT'])
df_season_composite['4FACTOR'] = df_season_composite['W4Factor'] * df_season_composite['WINPCT'] + df_season_composite['L4Factor'] * (1 - df_season_composite['WINPCT'])
df_season_composite['OFF_EFF'] = df_season_composite['WOffRtg'] * df_season_composite['WINPCT'] + df_season_composite['LOffRtg'] * (1 - df_season_composite['WINPCT'])
df_season_composite['DEF_EFF'] = df_season_composite['WDefRtg'] * df_season_composite['WINPCT'] + df_season_composite['LDefRtg'] * (1 - df_season_composite['WINPCT'])
df_season_composite['ASSIST_RATIO'] = df_season_composite['WAstR'] * df_season_composite['WINPCT'] + df_season_composite['LAstR'] * (1 - df_season_composite['WINPCT'])
df_season_composite['DEF_REB_PCT'] = df_season_composite['WDRP'] * df_season_composite['WINPCT'] + df_season_composite['LDRP'] * (1 - df_season_composite['WINPCT'])
df_season_composite['FT_PCT'] = df_season_composite['WFTPCT'] * df_season_composite['WINPCT'] + df_season_composite['LFTPCT'] * (1 - df_season_composite['WINPCT'])

df_season_composite.reset_index(inplace = True)

In [14]:
#Kentucy and Witchita State went undefeated causing problems with the data since cant calculate average stats without WINPCT

df_season_composite[df_season_composite['LOSSES'].isnull()]

,Season,WTeamID,WINS,LOSSES,WINPCT,WPIE,WeFGP,WTOR,WORP,WFTAR,W4Factor,WOffRtg,WDefRtg,WAstR,WDRP,WFTPCT,LPIE,LeFGP,LTOR,LORP,LFTAR,L4Factor,LOffRtg,LDefRtg,LAstR,LDRP,LFTPCT,PIE,FG_PCT,TURNOVER_RATE,OFF_REB_PCT,FT_RATE,4FACTOR,OFF_EFF,DEF_EFF,ASSIST_RATIO,DEF_REB_PCT,FT_PCT
55,2010,3163,32,nan,nan,0.928174,0.561939,0.176872,0.406115,0.305037,0.395972,1.200566,0.696032,0.188209,0.722764,0.717560,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
703,2012,3124,33,nan,nan,0.856320,0.517726,0.164713,0.421396,0.385422,0.390361,1.179306,0.779085,0.183237,0.695541,0.723836,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1423,2014,3163,34,nan,nan,0.897042,0.565791,0.145199,0.354695,0.263919,0.373143,1.241834,0.704262,0.207869,0.715950,0.759112,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1579,2014,3323,32,nan,nan,0.763409,0.554517,0.164881,0.403132,0.341372,0.394859,1.222760,0.860831,0.189619,0.692829,0.751518,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1947,2015,3343,29,nan,nan,0.797543,0.545809,0.179374,0.368518,0.303722,0.382429,1.154541,0.777819,0.173258,0.756462,0.684458,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2120,2016,3163,32,nan,nan,0.896457,0.582813,0.142666,0.395853,0.218501,0.380737,1.287244,0.700788,0.204482,0.731484,0.787582,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2467,2017,3163,28,nan,nan,0.841622,0.588118,0.150841,0.371130,0.282790,0.389602,1.275135,0.786337,0.218971,0.677548,0.777917,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2816,2018,3163,27,nan,nan,0.878578,0.604576,0.150400,0.383673,0.248881,0.393497,1.295404,0.734149,0.215517,0.724853,0.760509,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2932,2018,3280,29,nan,nan,0.795898,0.534819,0.132048,0.399114,0.317173,0.374339,1.254101,0.832290,0.158592,0.701938,0.738095,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [15]:
#Complete hack to fix the data

df_season_composite.loc[4064,'WINPCT'] = 1
df_season_composite.loc[4064,'LOSSES'] = 0
df_season_composite.loc[4064,'PIE'] = df_season_composite.loc[4064,'WPIE']
df_season_composite.loc[4064,'FG_PCT'] = df_season_composite.loc[4064,'WeFGP']
df_season_composite.loc[4064,'TURNOVER_RATE'] = df_season_composite.loc[4064,'WTOR']
df_season_composite.loc[4064,'OFF_REB_PCT'] = df_season_composite.loc[4064,'WORP']
df_season_composite.loc[4064,'FT_RATE'] = df_season_composite.loc[4064,'WFTAR']
df_season_composite.loc[4064,'4FACTOR'] = df_season_composite.loc[4064,'W4Factor']
df_season_composite.loc[4064,'OFF_EFF'] = df_season_composite.loc[4064,'WOffRtg']
df_season_composite.loc[4064,'DEF_EFF'] = df_season_composite.loc[4064,'WDefRtg']
df_season_composite.loc[4064,'ASSIST_RATIO'] = df_season_composite.loc[4064,'WAstR']
df_season_composite.loc[4064,'DEF_REB_PCT'] = df_season_composite.loc[4064,'WDRP']
df_season_composite.loc[4064,'FT_PCT'] = df_season_composite.loc[4064,'WFTPCT']

df_season_composite.loc[4211,'WINPCT'] = 1
df_season_composite.loc[4211,'LOSSES'] = 0
df_season_composite.loc[4211,'PIE'] = df_season_composite.loc[4211,'WPIE']
df_season_composite.loc[4211,'FG_PCT'] = df_season_composite.loc[4211,'WeFGP']
df_season_composite.loc[4211,'TURNOVER_RATE'] = df_season_composite.loc[4211,'WTOR']
df_season_composite.loc[4211,'OFF_REB_PCT'] = df_season_composite.loc[4211,'WORP']
df_season_composite.loc[4211,'FT_RATE'] = df_season_composite.loc[4211,'WFTAR']
df_season_composite.loc[4211,'4FACTOR'] = df_season_composite.loc[4211,'W4Factor']
df_season_composite.loc[4211,'OFF_EFF'] = df_season_composite.loc[4211,'WOffRtg']
df_season_composite.loc[4211,'DEF_EFF'] = df_season_composite.loc[4211,'WDefRtg']
df_season_composite.loc[4211,'ASSIST_RATIO'] = df_season_composite.loc[4211,'WAstR']
df_season_composite.loc[4211,'DEF_REB_PCT'] = df_season_composite.loc[4211,'WDRP']
df_season_composite.loc[4211,'FT_PCT'] = df_season_composite.loc[4211,'WFTPCT']

In [16]:
#we only need the final summary stats

df_season_composite.drop(['WINS','WPIE','WeFGP','WTOR','WORP','WFTAR','W4Factor','WOffRtg','WDefRtg','WAstR','WDRP','WFTPCT'], axis=1, inplace=True)
df_season_composite.drop(['LOSSES','LPIE','LeFGP','LTOR','LORP','LFTAR','L4Factor','LOffRtg','LDefRtg','LAstR','LDRP','LFTPCT'], axis=1, inplace=True)
df_season_composite.head()

,Season,WTeamID,WINPCT,PIE,FG_PCT,TURNOVER_RATE,OFF_REB_PCT,FT_RATE,4FACTOR,OFF_EFF,DEF_EFF,ASSIST_RATIO,DEF_REB_PCT,FT_PCT
0,2010.000000,3102.000000,0.035714,0.232527,0.407876,0.227935,0.345366,0.257720,0.327865,0.839445,1.166611,0.117555,0.608409,0.684589
1,2010.000000,3103.000000,0.566667,0.556495,0.441526,0.224153,0.391232,0.364156,0.365518,0.953100,0.913862,0.139969,0.634645,0.704329
2,2010.000000,3104.000000,0.379310,0.461117,0.436584,0.216921,0.352448,0.273776,0.340420,0.899547,0.941835,0.127039,0.636028,0.621109
3,2010.000000,3105.000000,0.518519,0.459657,0.437404,0.289757,0.384361,0.505554,0.400106,0.865314,0.924583,0.100359,0.609626,0.694134
4,2010.000000,3106.000000,0.413793,0.484470,0.377771,0.239378,0.389308,0.465138,0.358585,0.840086,0.869252,0.120614,0.671988,0.642189


In [17]:
#a little housekeeping to make easier to graph correlation matrix

columns = list(df_season_composite.columns.values) 
columns.pop(columns.index('WINPCT')) 
columns.append('WINPCT')
df_season_composite = df_season_composite[columns]
df_season_composite.rename(columns={'WTeamID':'TeamID'}, inplace=True)
df_season_composite.head()

,Season,TeamID,PIE,FG_PCT,TURNOVER_RATE,OFF_REB_PCT,FT_RATE,4FACTOR,OFF_EFF,DEF_EFF,ASSIST_RATIO,DEF_REB_PCT,FT_PCT,WINPCT
0,2010.000000,3102.000000,0.232527,0.407876,0.227935,0.345366,0.257720,0.327865,0.839445,1.166611,0.117555,0.608409,0.684589,0.035714
1,2010.000000,3103.000000,0.556495,0.441526,0.224153,0.391232,0.364156,0.365518,0.953100,0.913862,0.139969,0.634645,0.704329,0.566667
2,2010.000000,3104.000000,0.461117,0.436584,0.216921,0.352448,0.273776,0.340420,0.899547,0.941835,0.127039,0.636028,0.621109,0.379310
3,2010.000000,3105.000000,0.459657,0.437404,0.289757,0.384361,0.505554,0.400106,0.865314,0.924583,0.100359,0.609626,0.694134,0.518519
4,2010.000000,3106.000000,0.484470,0.377771,0.239378,0.389308,0.465138,0.358585,0.840086,0.869252,0.120614,0.671988,0.642189,0.413793


In [19]:
# Now lets do the same thing for the actual tourney

NCAATourney_SummaryW = pd.DataFrame()
NCAATourney_SummaryL = pd.DataFrame()
NCAATourney_SummaryW['WINS'] = df_tourney['WTeamID'].groupby([df_tourney['Season'], df_tourney['WTeamID']]).count()
NCAATourney_SummaryL['LOSSES'] = df_tourney['LTeamID'].groupby([df_tourney['Season'], df_tourney['LTeamID']]).count()
##NCAATourney_SummaryW['LOSSES'].fillna(0, inplace=True)
#NCAATourney_SummaryW['GAMES'] = NCAATourney_SummaryW['WINS'] + NCAATourney_SummaryW['LOSSES']
NCAATourney_SummaryW.reset_index(inplace = True)
NCAATourney_SummaryL.reset_index(inplace = True)
NCAATourney_SummaryW.rename(columns={'WTeamID':'TeamID'}, inplace=True)
NCAATourney_SummaryL.rename(columns={'LTeamID':'TeamID'}, inplace=True)

NCAATourney_Summary = pd.merge(left=NCAATourney_SummaryW, right=NCAATourney_SummaryL, how='outer', on=['Season', 'TeamID'])
NCAATourney_Summary['LOSSES'].fillna(0, inplace=True) 
NCAATourney_Summary['GAMES'] = NCAATourney_Summary['LOSSES'] + NCAATourney_Summary['WINS']

NCAATourney_Summary.head()

,Season,TeamID,WINS,LOSSES,GAMES
0,1998.000000,3104.000000,2.000000,1.000000,3.000000
1,1998.000000,3112.000000,2.000000,1.000000,3.000000
2,1998.000000,3116.000000,4.000000,1.000000,5.000000
3,1998.000000,3155.000000,1.000000,1.000000,2.000000
4,1998.000000,3161.000000,1.000000,1.000000,2.000000


In [25]:
#Get seeds of teams for all tourney games

df_seeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [26]:
# Convert string to an integer

df_seeds['seed_int'] = df_seeds['Seed'].apply( lambda x : int(x[1:3]) )
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) 
df_seeds.rename(columns={'seed_int':'Seed'},inplace=True)



NameError: name 'df_team_conferences_stage' is not defined